# Geocoding and Mapping of Fire and Police Departments: Colorado Focus

- Import and clean data from multiple sources.
- Merge datasets based on the common `agency_name` column to consolidate information.
- Identify and fill in missing data from the primary dataset using supplementary data sources.
- Geocode addresses to obtain latitude and longitude coordinates.
- Export the cleaned, organized, and geocoded data to CSV files for further analysis and reporting.

In [1]:
# Install required libraries with conda
# !conda install -y pandas geopy folium -c conda-forge

In [2]:
# Dependencies and Setup
import pandas as pd
from geopy.geocoders import GoogleV3
from geopy.distance import geodesic
import folium
from folium import plugins
from dotenv import load_dotenv
import os

load_dotenv()
google_api_key = 'my api creds' # Hard coding works

# os.environ.get('GOOGLE_MAPS_API_KEY') Using this causes problems

In [3]:
# Store filepath in a variable
police_data = 'resources/data/police_departments.csv'
fire_data = 'resources/data/usfa-registry-national.txt'
agency_addrs = 'resources/data/27681-0001-Data.txt'
co_leads =  'resources/data/co_leads.csv'

# Read each of the respective files (police, fire, agency_n, agency_addrs) and store into Pandas dataframe
police_data_df = pd.read_csv(police_data)
fire_data_df = pd.read_csv(fire_data, delimiter='\t', encoding='ISO-8859-1')
agency_addrs_df = pd.read_csv(agency_addrs, delimiter='\t', encoding='ISO-8859-1')
co_leads_df = pd.read_csv(co_leads)

In [4]:
# Show header for DataFrames
display(police_data_df.head(), fire_data_df.head(), agency_addrs_df.head(), co_leads_df.head())


,name,sworn,type,state,county,city,ftciv,ptciv,ptsworn,zip,fips
0,New York City Police Department,36023,Local police department,NY,New York,New York,13903,0,0,10038,36061
1,Chicago Police Dept,13354,Local police department,IL,Cook,Chicago,966,0,0,60653,17031
2,Los Angeles Police Department,9727,Local police department,CA,Los Angeles,Los Angeles,3272,0,0,90012,6037
3,Los Angeles County Sheriff's Office,9461,Sheriff's office,CA,Los Angeles,Monterey Park,7141,15,31,91754,6037
4,California Highway Patrol,7202,Primary state law enforcement agency,CA,Sacramento,Sacramento,3168,0,0,95818,6067


,FDID,Fire dept name,HQ addr1,HQ addr2,HQ city,HQ state,HQ zip,Mail addr1,Mail addr2,Mail PO box,...,Dept Type,Organization Type,Website,Number Of Stations,Active Firefighters - Career,Active Firefighters - Volunteer,Active Firefighters - Paid per Call,Non-Firefighting - Civilian,Non-Firefighting - Volunteer,Primary agency for emergency mgmt
0,61014,#9 Area Volunteer Fire Department,115 F Station LN,NaN,Eufaula,OK,74432,115 F Station LN,NaN,NaN,...,Volunteer,"Local (includes career, combination, and volun...",NaN,1,0,15,0,0.0,2.0,Yes
1,3010,101 Gamaliel Fire Protection District #5,5012 101 HWY,NaN,Gamaliel,AR,72537,NaN,NaN,PO Box 36,...,Volunteer,"Local (includes career, combination, and volun...",NaN,2,0,29,0,0.0,6.0,No
2,F3073,104th FW Barnes ANG Fire/Rescue,175 Falcon DR,104th FW Barnes ANGB,Westfield,MA,1085,175 Falcon DR,104th FW Barnes ANGB,NaN,...,Career,Federal government (Department of Defense fire...,NaN,1,30,0,0,0.0,0.0,No
3,36005,106 Rural Fire District,PO Box 154,NaN,Newkirk,OK,74647-0154,NaN,NaN,PO Box 154,...,Volunteer,"Local (includes career, combination, and volun...",NaN,1,0,8,0,0.0,2.0,NaN
4,4602,11 Point Rural Fire Association,908 E Main ST,NaN,Willow Springs,MO,65793,908 E Main ST,NaN,NaN,...,Volunteer,"Local (includes career, combination, and volun...",NaN,3,0,37,0,0.0,11.0,NaN


,CSLLEA08_ID,AGCYTYPE,SUBTYPE1,SUBTYPE2,TRIBAL,Q1A1,Q1A2,Q1A3,Q1A4,Q1A5,...,Q6_TOT,AGCYNAME,ADDR1,ADDR2,CITY,STATE,ZIP,STATENAME,COUNTY,FIPS
0,11000120,0,888,888,0,1,1,1,1,1,...,3,Haven Police Department,Po Box 356,,Haven,KS,67543,Kansas,Reno,20155
1,11000420,0,888,888,0,1,1,1,1,1,...,5,New Carlisle Police,113 South Arch Street,Post Office Box 6,New Carlisle,IN,46552,Indiana,St. Joseph,18141
2,11000730,0,888,888,0,1,1,1,1,1,...,10,Spring Grove Police Dept,7401 Meyer Road,,Spring Grove,IL,60081,Illinois,McHenry,17111
3,11000910,0,888,888,0,1,1,1,1,1,...,17,Lake Villa Police Dept,65 Cedar Ave,,Lake Villa,IL,60046,Illinois,Lake,17097
4,11001030,0,888,888,0,1,1,1,1,1,...,8,Clare Police Department,207 W Fifth St,,Clare,MI,48617,Michigan,Clare,26035


,agency_name,hq_addr1,hq_addr2,hq_city,hq_state,hq_zip
0,Adams County Fire Protection District,8055 N Washington ST,,Denver,CO,80229-5818
1,North Metro Fire Rescue District,101 Spader WAY,,Broomfield,CO,80020-2421
2,Thornton Fire Department,9500 Civic Center DR,,Thornton,CO,80229-4326
3,Westminster Fire Department,9110 Yates ST,,Westminster,CO,80031-2540
4,Aurora Fire Department,15151 E Alameda Pkwy #4100,,Aurora,CO,80012-1555


In [5]:
# Define dictionaries for renaming columns, and standardize column names that have same data
police_rename_dict = {
    'name': 'agency_name',
    'sworn' : 'sworn_active_persnl',
    'type' : 'agency_type'
}

fire_rename_dict = {
    'Fire dept name' : 'agency_name',
    'HQ addr1' : 'hq_addr1',
    'HQ addr2' : 'hq_addr2',
    'HQ city' : 'hq_city',
    'HQ state' : 'hq_state',
    'HQ zip' : 'hq_zip',
    'Mail addr1' : 'addr1',
    'Mail addr2' : 'addr2',
    'Mail PO box' : 'po_box',
    'Mail city' : 'city',
    'Mail state' : 'state',
    'Mail zip' : 'zip',
    'HQ phone' : 'hq_ph',
    'HQ fax' : 'hq_fax',
    'County' : 'county',
    'Dept Type' : 'agency_type',
    'Organization Type' : 'org_type',
    'Website' : 'website',
    'Active Firefighters - Career' : 'sworn_active_persnl'
}

addrs_rename_dict = {
    'AGCYNAME' : 'agency_name',
    'ADDR1' : 'addr1', 
    'ADDR2' : 'addr2',
    'CITY' : 'city',
    'STATE' : 'state',
    'ZIP' : 'zip',
    'COUNTY' : 'county'
}

# Rename columns in police_data_df
police_data_df.rename(columns=police_rename_dict, inplace=True)

# Rename columns in fire_data_df
fire_data_df.rename(columns=fire_rename_dict, inplace=True)

# Rename columns in agency_addrs_df
agency_addrs_df.rename(columns=addrs_rename_dict, inplace=True)

# Verify: Display the first few rows to verify the renaming
display(police_data_df.head(), fire_data_df.head(), agency_addrs_df.head())

,agency_name,sworn_active_persnl,agency_type,state,county,city,ftciv,ptciv,ptsworn,zip,fips
0,New York City Police Department,36023,Local police department,NY,New York,New York,13903,0,0,10038,36061
1,Chicago Police Dept,13354,Local police department,IL,Cook,Chicago,966,0,0,60653,17031
2,Los Angeles Police Department,9727,Local police department,CA,Los Angeles,Los Angeles,3272,0,0,90012,6037
3,Los Angeles County Sheriff's Office,9461,Sheriff's office,CA,Los Angeles,Monterey Park,7141,15,31,91754,6037
4,California Highway Patrol,7202,Primary state law enforcement agency,CA,Sacramento,Sacramento,3168,0,0,95818,6067


,FDID,agency_name,hq_addr1,hq_addr2,hq_city,hq_state,hq_zip,addr1,addr2,po_box,...,agency_type,org_type,website,Number Of Stations,sworn_active_persnl,Active Firefighters - Volunteer,Active Firefighters - Paid per Call,Non-Firefighting - Civilian,Non-Firefighting - Volunteer,Primary agency for emergency mgmt
0,61014,#9 Area Volunteer Fire Department,115 F Station LN,NaN,Eufaula,OK,74432,115 F Station LN,NaN,NaN,...,Volunteer,"Local (includes career, combination, and volun...",NaN,1,0,15,0,0.0,2.0,Yes
1,3010,101 Gamaliel Fire Protection District #5,5012 101 HWY,NaN,Gamaliel,AR,72537,NaN,NaN,PO Box 36,...,Volunteer,"Local (includes career, combination, and volun...",NaN,2,0,29,0,0.0,6.0,No
2,F3073,104th FW Barnes ANG Fire/Rescue,175 Falcon DR,104th FW Barnes ANGB,Westfield,MA,1085,175 Falcon DR,104th FW Barnes ANGB,NaN,...,Career,Federal government (Department of Defense fire...,NaN,1,30,0,0,0.0,0.0,No
3,36005,106 Rural Fire District,PO Box 154,NaN,Newkirk,OK,74647-0154,NaN,NaN,PO Box 154,...,Volunteer,"Local (includes career, combination, and volun...",NaN,1,0,8,0,0.0,2.0,NaN
4,4602,11 Point Rural Fire Association,908 E Main ST,NaN,Willow Springs,MO,65793,908 E Main ST,NaN,NaN,...,Volunteer,"Local (includes career, combination, and volun...",NaN,3,0,37,0,0.0,11.0,NaN


,CSLLEA08_ID,AGCYTYPE,SUBTYPE1,SUBTYPE2,TRIBAL,Q1A1,Q1A2,Q1A3,Q1A4,Q1A5,...,Q6_TOT,agency_name,addr1,addr2,city,state,zip,STATENAME,county,FIPS
0,11000120,0,888,888,0,1,1,1,1,1,...,3,Haven Police Department,Po Box 356,,Haven,KS,67543,Kansas,Reno,20155
1,11000420,0,888,888,0,1,1,1,1,1,...,5,New Carlisle Police,113 South Arch Street,Post Office Box 6,New Carlisle,IN,46552,Indiana,St. Joseph,18141
2,11000730,0,888,888,0,1,1,1,1,1,...,10,Spring Grove Police Dept,7401 Meyer Road,,Spring Grove,IL,60081,Illinois,McHenry,17111
3,11000910,0,888,888,0,1,1,1,1,1,...,17,Lake Villa Police Dept,65 Cedar Ave,,Lake Villa,IL,60046,Illinois,Lake,17097
4,11001030,0,888,888,0,1,1,1,1,1,...,8,Clare Police Department,207 W Fifth St,,Clare,MI,48617,Michigan,Clare,26035


In [6]:
# Drop named columns
police_data_df = police_data_df.drop(['ftciv', 'ptciv', 'ptsworn'], axis=1)
fire_data_df = fire_data_df.drop(['FDID', 'Number Of Stations', 'Active Firefighters - Volunteer', 'Active Firefighters - Paid per Call', 'Non-Firefighting - Civilian', 'Non-Firefighting - Volunteer', 'Primary agency for emergency mgmt'], axis=1)

#Save dataframe to a new file
police_data_df.to_csv(r'C:\Users\jchan\csi360_fire_police\resources\Output\police_data_clean_df.csv', index=False)
fire_data_df.to_csv(r'C:\Users\jchan\csi360_fire_police\resources\Output\fire_data_clean_df.csv', index=False)


In [7]:
# Select several columns to keep.
columns_to_select = ['agency_name', 'addr1', 'addr2']

# Select the specified columns
agency_addrs_df = agency_addrs_df[columns_to_select]

print(agency_addrs_df.head())

                agency_name                  addr1              addr2
0   Haven Police Department             Po Box 356                   
1       New Carlisle Police  113 South Arch Street  Post Office Box 6
2  Spring Grove Police Dept        7401 Meyer Road                   
3    Lake Villa Police Dept           65 Cedar Ave                   
4   Clare Police Department         207 W Fifth St                   


In [8]:
# For ease of commenting df1 is police_data_df and df2 is agency_addrs_df
# Test add column and iterate over rows to plug in missing data.
# Step 1: Add df2 columns to df1 with no data (initialize with empty strings)
for column in agency_addrs_df.columns:
    if column != 'agency_name':  # Avoid adding the common column again
        police_data_df[column] = ""

# For each row in df1, find matching agency_name in in df2  
# For each matching row in df2, fill in missing data in df1 columns
for index, row in police_data_df.iterrows():

    #Find df2 row where agency_name matches current row in df1
    matching_row = agency_addrs_df[agency_addrs_df['agency_name'] == row['agency_name']]
    
    if not matching_row.empty:
        # Step 3: Insert corresponding data into df1
        for column in agency_addrs_df.columns:
            if column != 'agency_name':  # Skip the common column
                police_data_df.at[index, column] = matching_row.iloc[0][column]

# Display the updated df1
print(police_data_df.head())

                           agency_name  sworn_active_persnl  \
0      New York City Police Department                36023   
1                  Chicago Police Dept                13354   
2        Los Angeles Police Department                 9727   
3  Los Angeles County Sheriff's Office                 9461   
4            California Highway Patrol                 7202   

                            agency_type state       county           city  \
0               Local police department    NY     New York       New York   
1               Local police department    IL         Cook        Chicago   
2               Local police department    CA  Los Angeles    Los Angeles   
3                      Sheriff's office    CA  Los Angeles  Monterey Park   
4  Primary state law enforcement agency    CA   Sacramento     Sacramento   

     zip   fips                        addr1                 addr2  
0  10038  36061  One Police Plaza, Room 1400                        
1  60653  17031     

In [9]:
# Sort DataFrame by agency_name in alphabetical order (A to Z)
police_data_sorted = police_data_df.sort_values(by='agency_name', ascending=True)

# Display the sorted DataFrame
display(police_data_sorted.head())

,agency_name,sworn_active_persnl,agency_type,state,county,city,zip,fips,addr1,addr2
14109,1854 Treaty Authority Tribal Enforcement Division,3,Special jurisdiction,MN,St. Louis,Duluth,55811,27137,4428 Haines Road,
17298,34,0,Local police department,Boulder City,Nevada,NaN,NV,Clark,,
4677,Abbeville County Sheriff's Office,25,Sheriff's office,SC,Abbeville,Abbeville,29620,45001,Po Box 9,
5836,Abbeville Police Department,19,Local police department,SC,Abbeville,Abbeville,29620,45001,304 Charity St.,
8959,Abbeville Police Department,10,Local police department,AL,Henry,Abbeville,36310,1067,304 Charity St.,


In [10]:
# Concatenate the DataFrames vertically
merged_df = pd.concat([police_data_df, fire_data_df], ignore_index=True)

# Replace NaN values with a blank space if needed
merged_df.fillna(" ", inplace=True)

# Display the merged DataFrame
print(merged_df.head())


                           agency_name  sworn_active_persnl  \
0      New York City Police Department                36023   
1                  Chicago Police Dept                13354   
2        Los Angeles Police Department                 9727   
3  Los Angeles County Sheriff's Office                 9461   
4            California Highway Patrol                 7202   

                            agency_type state       county           city  \
0               Local police department    NY     New York       New York   
1               Local police department    IL         Cook        Chicago   
2               Local police department    CA  Los Angeles    Los Angeles   
3                      Sheriff's office    CA  Los Angeles  Monterey Park   
4  Primary state law enforcement agency    CA   Sacramento     Sacramento   

     zip   fips                        addr1                 addr2 hq_addr1  \
0  10038  36061  One Police Plaza, Room 1400                                  


In [11]:
# List the columns in the desired order
new_column_order = [
    'agency_name', 'agency_type', 'sworn_active_persnl', 'hq_addr1', 'hq_addr2',
    'hq_city', 'hq_state', 'hq_zip', 'addr1', 'addr2', 'po_box', 'city', 'state', 
    'county', 'zip', 'hq_ph', 'hq_fax', 'org_type', 'website', 'fips'
]
# Reorder the DataFrame columns
merged_df = merged_df[new_column_order]

# Display the DataFrame to check the new order
print(merged_df.head())

                           agency_name                           agency_type  \
0      New York City Police Department               Local police department   
1                  Chicago Police Dept               Local police department   
2        Los Angeles Police Department               Local police department   
3  Los Angeles County Sheriff's Office                      Sheriff's office   
4            California Highway Patrol  Primary state law enforcement agency   

   sworn_active_persnl hq_addr1 hq_addr2 hq_city hq_state hq_zip  \
0                36023                                             
1                13354                                             
2                 9727                                             
3                 9461                                             
4                 7202                                             

                         addr1                 addr2 po_box           city  \
0  One Police Plaza, Room 1400  

In [12]:
#Save merged dataframe to a new file
merged_df.to_csv(r'C:\Users\jchan\csi360_fire_police\resources\Output\merged_data2.csv', index=False)


## Geocode Colorado Fire Deptartment Addresses


In [13]:
# Debug google maps api
print(google_api_key)


REDACTED


In [14]:
# Initialize geolocator. 
geolocator = GoogleV3(api_key=google_api_key)


In [15]:
# Function to get latitude and longitude from address
def get_lat_long(address):
    try:
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return None, None

In [16]:
# Test with a known address
location = geolocator.geocode("1600 Pennsylvania Ave NW, Washington, DC 20500")
print(location.latitude, location.longitude)

38.89764 -77.0363948


In [17]:
# Combine address fields and get latitude/longitude
co_leads_df['Full_Address'] = co_leads_df['hq_addr1'] + ', ' + co_leads_df['hq_city'] + ', ' + co_leads_df['hq_state'] + ' ' + co_leads_df['hq_zip'].astype(str)
co_leads_df['Latitude'], co_leads_df['Longitude'] = zip(*co_leads_df['Full_Address'].apply(get_lat_long))
print(co_leads_df.head())

                             agency_name                    hq_addr1 hq_addr2  \
0  Adams County Fire Protection District        8055 N Washington ST            
1       North Metro Fire Rescue District              101 Spader WAY            
2               Thornton Fire Department        9500 Civic Center DR            
3            Westminster Fire Department               9110 Yates ST            
4                 Aurora Fire Department  15151 E Alameda Pkwy #4100            

       hq_city hq_state      hq_zip  \
0       Denver       CO  80229-5818   
1   Broomfield       CO  80020-2421   
2     Thornton       CO  80229-4326   
3  Westminster       CO  80031-2540   
4       Aurora       CO  80012-1555   

                                        Full_Address   Latitude   Longitude  
0        8055 N Washington ST, Denver, CO 80229-5818  39.735872 -104.978725  
1          101 Spader WAY, Broomfield, CO 80020-2421  39.916333 -105.067569  
2      9500 Civic Center DR, Thornton, CO 80

In [18]:
# CSI360 location
csi360_location = (39.0911, -104.8726)  # Approximate lat/long for 590 Highway 105 Suite # 276, Monument, CO 80132

# Function to check if within 30 miles
def within_30_miles(row):
    try:
        department_location = (row['Latitude'], row['Longitude'])
        return geodesic(csi360_location, department_location).miles <= 30
    except:
        return False

# Apply the function to filter departments within and outside of 30 miles
co_leads_df['Within_30_Miles'] = co_leads_df.apply(within_30_miles, axis=1)


In [19]:
# Separate the dataframes for departments within and outside the 30-mile radius
within_30_df = co_leads_df[co_leads_df['Within_30_Miles'] == True]
outside_30_df = co_leads_df[co_leads_df['Within_30_Miles'] == False]

# Save the filtered dataframes to CSV files
within_30_df.to_csv('C:/Users/jchan/csi360_fire_police/resources/Output/co_fd_within_30_miles.csv', index=False)
outside_30_df.to_csv('C:/Users/jchan/csi360_fire_police/resources/Output/co_fd_outside_30_miles.csv', index=False)

## Create Map with Overlays

In [21]:
# Create base map centered on CSI360 location
m = folium.Map(location=csi360_location, zoom_start=10)

# Add a circular overlay for within 30 miles
folium.Circle(
    location=csi360_location,
    radius=30*1609.34,  # 30 miles in meters
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.2,
    popup='Within 30 Miles'
).add_to(m)

# Add fire department markers
for _, row in within_30_df.iterrows():
    folium.Marker(
        location=(row['Latitude'], row['Longitude']),
        popup=f"{row['agency_name']}<br>{row['Full_Address']}",
        icon=folium.Icon(color='blue')
    ).add_to(m)

for _, row in outside_30_df.iterrows():
    folium.Marker(
        location=(row['Latitude'], row['Longitude']),
        popup=f"{row['agency_name']}<br>{row['Full_Address']}",
        icon=folium.Icon(color='red')
    ).add_to(m)

# Add a layer control to toggle overlays
folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('C:/Users/jchan/csi360_fire_police/resources/Output/colorado_fire_departments_map.html')
